## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Видим, что столбец 'id' не принесет никакой пользы при обучении модели, поэтому следует его удалить.

In [6]:
# соберем в список все датасеты,чтобы пройтись по ним циклом
data_all = [
    data_0.drop('id', axis=1),
    data_1.drop('id', axis=1),
    data_2.drop('id', axis=1)
]

In [7]:
for i in range(len(data_all)):
    print('Общая информация по датасету', i)
    print(data_all[i].info())
    print()

Общая информация по датасету 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None

Общая информация по датасету 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB
None

Общая информация по датасету 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Nu

In [8]:
for i in range(len(data_all)):
    print('Статистика по столбцам датасета', i)
    print(data_all[i].describe())
    print()

Статистика по столбцам датасета 0
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

Статистика по столбцам датасета 1
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%    

In [9]:
print(data_0.duplicated().sum(), data_1.duplicated().sum(), data_2.duplicated().sum())

0 0 0


Изучение датасетов показало, что в таблицах нет пропусков и дубликатов. Так как данные синтетические и мы не знаем, что означают столбцы с признаками, судить о выбросах в данных мы не можем. Все столбцы с признаками содержат в себе количественные переменные. Целевым признаком является столбец 'product'.

## Обучение и проверка модели

In [10]:
# создадим пустые списки, чтобы сохранить в них предсказания и правильные ответы
predictions = []
targets = []

# напишем цикл, который пройдет по каждому региону
for i in range(len(data_all)):
    features = data_all[i].drop(['product'], axis=1)
    target = data_all[i]['product']
    
    # разобьем данные на обучающую и валидационную выборки в соотношении 75:25
    features_train, features_valid, target_train, target_valid = (
        train_test_split(features, target, test_size=0.25, random_state=12345))
    
    # обучим модель и сделаем предсказания на валидационной выборке
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    
    # cохраним предсказания и правильные ответы на валидационной выборке
    predictions.append(predictions_valid)
    targets.append(target_valid)
    
    
    # напечатаем на экране средний запас предсказанного сырья и RMSE модели
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    print('Cредний запас предсказанного сырья в регионе', i, '=', predictions_valid.mean())
    print('Реальный средний запас сырья в регионе', i, '=', target_valid.mean())    
    print('RMSE модели', result)
    print()

Cредний запас предсказанного сырья в регионе 0 = 92.59256778438035
Реальный средний запас сырья в регионе 0 = 92.07859674082927
RMSE модели 37.5794217150813

Cредний запас предсказанного сырья в регионе 1 = 68.728546895446
Реальный средний запас сырья в регионе 1 = 68.72313602435997
RMSE модели 0.893099286775617

Cредний запас предсказанного сырья в регионе 2 = 94.96504596800489
Реальный средний запас сырья в регионе 2 = 94.88423280885438
RMSE модели 40.02970873393434



In [11]:
predictions_0 = pd.Series(predictions[0])
predictions_1 = pd.Series(predictions[1])
predictions_2 = pd.Series(predictions[2])
print(predictions_0.head(), predictions_1.head(), predictions_2.head())

0    95.894952
1    77.572583
2    77.892640
3    90.175134
4    70.510088
dtype: float64 0    82.663314
1    54.431786
2    29.748760
3    53.552133
4     1.243856
dtype: float64 0     93.599633
1     75.105159
2     90.066809
3    105.162375
4    115.303310
dtype: float64


In [12]:
target_0 = targets[0].reset_index(drop=True)
target_1 = targets[1].reset_index(drop=True)
target_2 = targets[2].reset_index(drop=True)
print(target_0.head(), target_1.head(), target_2.head())

0     10.038645
1    114.551489
2    132.603635
3    169.072125
4    122.325180
Name: product, dtype: float64 0    80.859783
1    53.906522
2    30.132364
3    53.906522
4     0.000000
Name: product, dtype: float64 0     61.212375
1     41.850118
2     57.776581
3    100.053761
4    109.897122
Name: product, dtype: float64


Изучив полученные с помощью линейной регрессии результаты, можно выделить модель, обученную на данных второго региона. Разительно отличается значение корня из средней квадратичной ошибки, оно равно 0,89 и означает, что правильные ответы незначительно отличаются от предсказаний модели. 

## Подготовка к расчёту прибыли

В условиях задачи сказано:
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [13]:
# введем константы
POINTS = 500
BEST_POINTS = 200
BUDGET_PER_REGION = 10000000000
REVENUE_PER_BARREL = 450000

In [14]:
enough_raw_material = BUDGET_PER_REGION/REVENUE_PER_BARREL/BEST_POINTS
print('Достаточный объём сырья для безубыточной разработки новой скважины:', round(enough_raw_material, 2), 'тыс. баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 тыс. баррелей


In [15]:
for i in range(len(data_all)):
    print('Статистика по объему запасов нефти (тыс. баррелей) в скважинах региона', i)
    print(data_all[i]['product'].describe())
    print()

Статистика по объему запасов нефти (тыс. баррелей) в скважинах региона 0
count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64

Статистика по объему запасов нефти (тыс. баррелей) в скважинах региона 1
count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64

Статистика по объему запасов нефти (тыс. баррелей) в скважинах региона 2
count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64



In [16]:
# посмотрим сколько процентов скважин в каждом регионе 
# содержат сырья больше, чем достаточный нам объем сырья
print(data_0['product'].quantile(0.65))
print(data_1['product'].quantile(0.85))
print(data_2['product'].quantile(0.62))

113.16725066331651
134.76630515769844
111.33727624548305


Средний объём запасов в скважинах во всех регионах ниже, чем достаточный объём сырья для безубыточной разработки новой скважины. В первом регионе 35% скважин содержат сырья больше, чем нам достаточно, во втором регионе - лишь 15% и в третьем - 38%.

In [17]:
# напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(targ, preds, count):
    # для удобства, сначала отсортируем предсказания по убыванию
    preds_sorted = preds.sort_values(ascending=False)
    
    # выберем скважины с максимальными значениями предсказаний
    selected = targ[preds_sorted.index][:count]
    
    # просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям
    target_volume_of_raw_materials = selected.sum()
    
    # рассчитаем прибыль для полученного объёма сырья
    return target_volume_of_raw_materials * REVENUE_PER_BARREL - BUDGET_PER_REGION

In [18]:
# проверим работоспособность функции
print("Прибыль с лучших 200 скважин в первом регионе:", round(profit(target_0, predictions_0, BEST_POINTS), 2))
print("Прибыль с лучших 200 скважин во втором регионе:", round(profit(target_1, predictions_1, BEST_POINTS), 2))
print("Прибыль с лучших 200 скважин в третьем регионе:", round(profit(target_2, predictions_2, BEST_POINTS), 2))

Прибыль с лучших 200 скважин в первом регионе: 3320826043.14
Прибыль с лучших 200 скважин во втором регионе: 2415086696.68
Прибыль с лучших 200 скважин в третьем регионе: 2710349963.6


## Расчёт прибыли и рисков 

In [19]:
state = RandomState(12345)

In [20]:
values_0 = []
for i in range(1000):
    target_subsample = target_0.sample(n=POINTS, replace=True, random_state=state)
    preds_subsample = predictions_0[target_subsample.index] 
    result = profit(target_subsample, preds_subsample, BEST_POINTS)
    values_0.append(result)
    
values_0 = pd.Series(values_0)

print('Средняя прибыль в первом регионе =', round(values_0.mean(), 2))

lower_0 = values_0.quantile(0.025)
upper_0 = values_0.quantile(0.975)
print("95%-ый доверительный интервал:", (lower_0, upper_0))

risk_0 = (values_0 < 0).mean()*100
print('Риск убытков составляет', risk_0, '%')

Средняя прибыль в первом регионе = 425938526.91
95%-ый доверительный интервал: (-102090094.83793654, 947976353.3583689)
Риск убытков составляет 6.0 %


In [21]:
values_1 = []
for i in range(1000):
    target_subsample = target_1.sample(n=POINTS, replace=True, random_state=state)
    preds_subsample = predictions_1[target_subsample.index] 
    result = profit(target_subsample, preds_subsample, BEST_POINTS)
    values_1.append(result)
    
values_1 = pd.Series(values_1)

print('Средняя прибыль во втором регионе =', round(values_1.mean(), 2))

lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
print("95%-ый доверительный интервал:", (lower_1, upper_1))

risk_1 = (values_1 < 0).mean()*100
print('Риск убытков составляет', risk_1, '%')

Средняя прибыль во втором регионе = 518259493.7
95%-ый доверительный интервал: (128123231.43308444, 953612982.0669085)
Риск убытков составляет 0.3 %


In [22]:
values_2 = []
for i in range(1000):
    target_subsample = target_2.sample(n=POINTS, replace=True, random_state=state)
    preds_subsample = predictions_2[target_subsample.index] 
    result = profit(target_subsample, preds_subsample, BEST_POINTS)
    values_2.append(result)
    
values_2 = pd.Series(values_2)

print('Средняя прибыль в третьем регионе =', round(values_2.mean(), 2))

lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
print("95%-ый доверительный интервал:", (lower_2, upper_2))

risk_2 = (values_2 < 0).mean()*100
print('Риск убытков составляет', risk_2, '%')

Средняя прибыль в третьем регионе = 420194005.34
95%-ый доверительный интервал: (-115852609.16001143, 989629939.8445739)
Риск убытков составляет 6.2 %


## Вывод

Было произведено исследование с целью выбора локации для скважины среди трех регионов. В ходе исследования, с помощью модели линейная регрессия были получены метрики для трех регионов, среди которых наиболее выделяется второй регион с наименьшей средней квадратичной ошибкой (корень из нее составил 0.89). 

Для безубыточной разработки скважины необходимо, чтобы объем сырья в скважине был не менее 111.11 тыс. баррелей. В этом плане выделяется третий регион, где 38% скважин сожержат объем сырья больший, чем достаточный. В первом регионе 35% скважин, во втором регионе - лишь 15%.

После применения техники Bootstrap с 1000 выборок, наименее рискованным показал себя второй регион, здесь же и наиболее высокий показатель средней прибыли 518.26 млрд. руб. Всвязи с этим, в качестве локации для скважины предлагается второй регион.